# function to find the best model by making and testing random models

In [13]:
import pandas as pd

from ucimlrepo import fetch_ucirepo

# fetch dataset
superconductivty_data = fetch_ucirepo(id=464)

# data (as pandas dataframes)
X = superconductivty_data.data.features
y = superconductivty_data.data.targets

df = X.join(y)

df.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.0
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.0


In [30]:
# Code from previous file

import tensorflow as tf

@tf.function # This decorator compiles the function to XLA/Graph (makes it even faster)
def train_step(
    model: tf.keras.Model, 
    X_tensor: tf.Tensor, 
    y_tensor: tf.Tensor, 
    learning_rate: float
) -> float:
    # Defensive Casting: Ensure inputs are float32 no matter what was passed in
    X_tensor = tf.cast(X_tensor, dtype=tf.float32)
    y_tensor = tf.cast(y_tensor, dtype=tf.float32)
    
    with tf.GradientTape() as tape:
        # Forward Pass
        predictions = model(X_tensor, training=True) 

        # Fix Shapes
        predictions = tf.reshape(predictions, [-1])
        
        # Calculate Loss
        loss = tf.math.reduce_mean(tf.math.square(predictions - y_tensor))

    # Backward Pass
    gradients = tape.gradient(loss, model.trainable_variables)

    # Manual Update
    for param, grad in zip(model.trainable_variables, gradients):
        if grad is not None:
            param.assign_sub(grad * learning_rate)
            
    return loss

def reset_weights(model):
    import tensorflow as tf
    for layer in model.layers:
        # Reset Kernel (Weights)
        if hasattr(layer, 'kernel_initializer') and hasattr(layer, 'kernel'):
            layer.kernel.assign(layer.kernel_initializer(tf.shape(layer.kernel)))
        
        # Reset Bias
        if hasattr(layer, 'bias_initializer') and hasattr(layer, 'bias'):
            layer.bias.assign(layer.bias_initializer(tf.shape(layer.bias)))
            
        # Reset other variables (like LSTM states if you had them)
        if hasattr(layer, 'recurrent_initializer') and hasattr(layer, 'recurrent_kernel'):
            layer.recurrent_kernel.assign(layer.recurrent_initializer(tf.shape(layer.recurrent_kernel)))

def train(model,
    X_train,
    y_train,
    epochs,
    learning_rate=0.01,
    batch_size=32,
    warm_start: bool = True
):
    import math

    if not warm_start:
        print("Cold Start: Resetting model weights to random...")
        reset_weights(model)
    else:
        print("Warm Start: Continuing training with existing weights...")

    df_train = X_train.join(y_train)
    num_samples = len(df_train)
    total_batches = math.ceil(num_samples / batch_size)
    best_loss: int = None
    best_params: list = None
    
    for i in range(epochs):
        print(f"--- Starting Epoch {i+1}/{epochs} ---")
        shuffled_df = df_train.sample(frac=1)
        X_shuffled = shuffled_df.iloc[:,:-1]
        y_shuffled = shuffled_df.iloc[:,-1]

        for j in range(0, num_samples, batch_size):
            current_batch = (j // batch_size) + 1
            
            # 1. Get the batch as Pandas objects
            X_batch_df = X_shuffled.iloc[j : j + batch_size]
            y_batch_series = y_shuffled.iloc[j : j + batch_size]

            # 2. Convert batch to tensors
            X_tensor = tf.convert_to_tensor(X_batch_df.values, dtype=tf.float32)
            y_tensor = tf.convert_to_tensor(y_batch_series.values, dtype=tf.float32)

            # 3. Pass Tensors to the function
            loss = train_step(model, X_tensor, y_tensor, learning_rate)
            
            # (Convert loss back to a regular number for printing
            print(f"  Batch {current_batch}/{total_batches} - Loss: {loss.numpy():.4f}", end="\r", flush=True)
        
            if best_loss == None:
                best_loss = loss
                best_params = model.get_weights()
            elif loss < best_loss:
                best_loss = loss
                best_params = model.get_weights()
            
            if loss == float('nan'):
                print()
                print("Training resulted in overflow of loss")
                print()
                print(f"best loss {best_loss}")
                return best_params

        print() 

    print("Training Complete.")
    print()
    print(f"best loss {best_loss}")
    return best_params

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Split the SCALED data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=0)

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GaussianDropout

In [21]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout

class ModelArchitecture:
    def __init__(self, layers_list: list[dict] = None):
        """
        Initialize with a list of dictionaries. Each dict represents a layer.
        Example:
        [
            {'type': 'dense', 'units': 64, 'activation': 'relu'},
            {'type': 'dropout', 'rate': 0.5},
            {'type': 'dense', 'units': 1, 'activation': 'linear'}
        ]
        """
        # Define the expected columns
        self.expected_cols = ['type', 'units', 'activation', 'rate']
        
        if layers_list:
            self.df = pd.DataFrame(layers_list)
            # Ensure all columns exist (fill missing with None/NaN)
            for col in self.expected_cols:
                if col not in self.df.columns:
                    self.df[col] = None
        else:
            # Create empty DF with correct columns
            self.df = pd.DataFrame(columns=self.expected_cols)

    def add_layer(self, layer_type, units=None, activation=None, rate=None):
        """Helper to add a row to the dataframe"""
        new_row = {
            'type': layer_type,
            'units': units,
            'activation': activation,
            'rate': rate
        }
        # Append logic (concat is preferred over append in new pandas)
        self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index=True)

    def get_layer_info(self, index):
        return self.df.iloc[index]

    def build_model(self, input_shape):
        """
        Iterates through the DataFrame and constructs the Keras model.
        """
        model = Sequential()
        
        for i, row in self.df.iterrows():
            layer_type = row['type']
            
            # Handle First Layer (needs input_shape)
            kwargs = {}
            if i == 0:
                kwargs['input_shape'] = input_shape

            # Add Layer
            match layer_type:
                case 'dense':
                    # int() conversion is needed because pandas usually stores numbers as floats
                    units = int(row['units']) 
                    act = row['activation']
                    model.add(Dense(units, activation=act, **kwargs))

                case 'dropout':
                    rate = row['rate']
                    model.add(Dropout(rate=rate))

                case 'gaussiandropout':
                    rate = row['rate']
                    model.add(GaussianDropout(rate=rate))
            
        return model
    
    def set_random_arch(
        self,
        num_layers,
        layer_types = ['dense'],
        dropout_layer_types = ['dropout'],
        dropout_chance = 0.5,
        min_dropout_rate = 0.1,
        max_dropout_rate = 0.5,
        min_units = 8,
        max_units = 64,
        activation = 'relu'
    ):
        self.__init__()
        
        # Add hidden layers
        for i in range(num_layers - 1):
            self.add_layer(
                layer_type = random.choice(layer_types),
                units = random.randint(min_units, max_units),
                activation =  activation
            )
            # Randomly add dropout after each layer
            if random.random() < dropout_chance:
                self.add_layer(
                    layer_type = random.choice(dropout_layer_types),
                    rate = random.uniform(min_dropout_rate, max_dropout_rate)
                )
        
        # Add output layer
        self.add_layer(
            layer_type = 'dense',
            units = 1, 
            activation = 'linear'
        )

        return self


In [31]:
import random
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def evaluate_candidate(
    X,
    y,
    test_number,
    num_layers,
    layer_types = ['dense'],
    dropout_layer_types = ['dropout'],
    dropout_chance = 0.5,
    min_dropout_rate = 0.1,
    max_dropout_rate = 0.5,
    min_units = 8,
    max_units = 64,
    activation = 'relu',
    epochs = 16
):
    
    architecture: ModelArchitecture = ModelArchitecture().set_random_arch(
        num_layers = num_layers,
        layer_types = layer_types,
        dropout_layer_types=dropout_layer_types,
        dropout_chance = dropout_chance,
        min_dropout_rate = min_dropout_rate,
        max_dropout_rate = max_dropout_rate,
        min_units = min_units,
        max_units = max_units,
        activation = activation
    )
    print(f"{num_layers} layers, attempt #{test_number+1}")

    # Prepare data
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=0)
    
    # Reshape Y for evaluation
    y_test_reshaped = y_test.values.reshape(-1, 1)


    input_dim = X_train.shape[1] 
    model = architecture.build_model(input_shape=(input_dim,))

    train(
        model=model,
        X_train=X_train,
        y_train=y_train,
        epochs=32,
        learning_rate=0.0005,
        warm_start=False
    )

    model.compile(loss='mean_squared_error', optimizer='adam')
    loss = model.evaluate(X_test, y_test_reshaped, verbose=0)
    rmse = loss**0.5

    return rmse, architecture

def find_best_model(
    X,
    y,
    num_tries = 4, # for each number of layers in range
    min_layers = 3,
    max_layers = 12,
    min_units = 8,
    max_units = 64,
    layer_types = ['dense'],
    dropout_layer_types = ['dropout'],
    dropout_chance = 0.5,
    min_dropout_rate = 0.1,
    max_dropout_rate = 0.5,
    activation = 'relu',
    epochs = 16
):
    if min_layers>max_layers or type(min_layers)!=int or type(max_layers)!=int:
        print("ERROR: min_layers must be less than max_layers and both must be integers")
        return
    
    # Flatten the experimental space
    tasks = [(l, t) for l in range(min_layers, max_layers + 1) for t in range(num_tries)]
    
    # Test all the models
    # Runs parallel
    # WARNING: TensorFlow and joblib (multiprocessing) often fight over the GPU.
    # If this crashes, change n_jobs to 1.
    results = Parallel(n_jobs=2)(
        delayed(evaluate_candidate)(
            X = X,
            y = y,
            num_layers = layers,
            test_number = test_number,
            layer_types = layer_types,
            dropout_layer_types = dropout_layer_types,
            dropout_chance = dropout_chance,
            min_dropout_rate = min_dropout_rate,
            max_dropout_rate = max_dropout_rate,
            min_units = min_units,
            max_units = max_units,
            activation = activation,
            epochs = epochs
        )
        for layers, test_number in tasks
    )

    # Find and return best model
    best_rmse = float('inf')
    best_architecture = None

    for rmse, architecture in results:
        if rmse < best_rmse:
            best_rmse = rmse
            best_architecture = architecture
    
    print()
    print()
    print(f"Overall Best RMSE {best_rmse}")
    return best_architecture


In [32]:
best_model = find_best_model(
    X = X,
    y = y,
    num_tries = 6
)

2025-12-18 21:25:12.446419: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-18 21:25:12.446419: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-18 21:25:12.454158: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-18 21:25:12.454185: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-18 21:25:12.824481: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized

3 layers, attempt #1
3 layers, attempt #2
Cold Start: Resetting model weights to random...
Cold Start: Resetting model weights to random...
--- Starting Epoch 1/32 ---
--- Starting Epoch 1/32 ---


/home/julian/StudioProjects/wade-datascience/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/julian/StudioProjects/wade-datascience/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-12-18 21:25:15.297674: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-12-18 21:25:15.297674: E external/local_xla/xla/stream_executor/cuda/c

  Batch 499/499 - Loss: 209.3341  Batch 476/499 - Loss: 462.0025
--- Starting Epoch 2/32 ---
  Batch 54/499 - Loss: 367.73831
--- Starting Epoch 2/32 ---
  Batch 499/499 - Loss: 160.22021
--- Starting Epoch 3/32 ---
  Batch 132/499 - Loss: 101.81241
--- Starting Epoch 3/32 ---
  Batch 362/499 - Loss: 186.5995  Batch 478/499 - Loss: 200.1263
--- Starting Epoch 4/32 ---
  Batch 499/499 - Loss: 110.2911
--- Starting Epoch 4/32 ---
  Batch 499/499 - Loss: 124.75143
--- Starting Epoch 5/32 ---
  Batch 499/499 - Loss: 224.4944 Batch 397/499 - Loss: 293.4150
--- Starting Epoch 5/32 ---
  Batch 499/499 - Loss: 139.8609  Batch 487/499 - Loss: 189.4792
--- Starting Epoch 6/32 ---
  Batch 499/499 - Loss: 293.5515  Batch 23/499 - Loss: 329.7499
--- Starting Epoch 6/32 ---
  Batch 499/499 - Loss: 99.35755  Batch 327/499 - Loss: 287.0806
--- Starting Epoch 7/32 ---
  Batch 88/499 - Loss: 112.87605  Batch 28/499 - Loss: 116.9860
--- Starting Epoch 7/32 ---
  Batch 499/499 - Loss: 202.42607 Batch 231/

/tmp/ipykernel_3420585/2545894807.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/home/julian/StudioProjects/wade-datascience/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Import object at 0x76c62af324d0>
To silence this warning, decorate the function with @tf.autograph

...

  Batch 499/499 - Loss: 1668452411789965456871952522149888.000031267888946861439975424000.0000
--- Starting Epoch 30/32 ---
  Batch 499/499 - Loss: 614711235108164835626973990486016.0000  Batch 352/499 - Loss: nan
--- Starting Epoch 31/32 ---
  Batch 499/499 - Loss: 226479331305640734724582813990912.0000
--- Starting Epoch 32/32 ---
  Batch 499/499 - Loss: nan91651185232913734425462702080.0000
--- Starting Epoch 2/32 ---
  Batch 499/499 - Loss: 83442187779244227385567750389760.00000  Batch 281/499 - Loss: nan
Training Complete.

best loss 383.04071044921875
  Batch 499/499 - Loss: nan
--- Starting Epoch 3/32 ---
12 layers, attempt #6s: nan


/tmp/ipykernel_3420585/2545894807.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/home/julian/StudioProjects/wade-datascience/.venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Cold Start: Resetting model weights to random...
--- Starting Epoch 1/32 ---


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Import object at 0x7608b4b35290>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


  Batch 499/499 - Loss: nan
--- Starting Epoch 4/32 ---
  Batch 499/499 - Loss: nan3.7744h 259/499 - Loss: 1276.2351
--- Starting Epoch 5/32 ---
  Batch 499/499 - Loss: 1275.3300h 484/499 - Loss: 1256.6343
--- Starting Epoch 2/32 ---
  Batch 499/499 - Loss: nan
--- Starting Epoch 6/32 ---
  Batch 499/499 - Loss: nan.95874h 205/499 - Loss: 458.4110
--- Starting Epoch 7/32 ---
  Batch 499/499 - Loss: 461.6823h 475/499 - Loss: 313.3132
--- Starting Epoch 3/32 ---
  Batch 494/499 - Loss: nan.91585h 313/499 - Loss: 221.7998
--- Starting Epoch 4/32 ---
  Batch 499/499 - Loss: nan
--- Starting Epoch 8/32 ---
  Batch 499/499 - Loss: 148.7699  Batch 340/499 - Loss: nan
--- Starting Epoch 5/32 ---
  Batch 499/499 - Loss: nan8475
--- Starting Epoch 9/32 ---
  Batch 499/499 - Loss: 104.56614 Batch 365/499 - Loss: nan
--- Starting Epoch 6/32 ---
  Batch 48/499 - Loss: 203.1900
--- Starting Epoch 10/32 ---
  Batch 499/499 - Loss: 135.63948
--- Starting Epoch 7/32 ---
  Batch 499/499 - Loss: nan.5446